In [1]:
import csv
import re
import string

In [2]:
from timeit import default_timer as timer

In [ ]:
def preprocess_text(text):
    """
    """
    text = text.strip().lower()
    text = re.sub('\d+|\d+\.\d+', '[NUM]', text)

    # PHI tags are in format [**INFORMATION**]
    text = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', text)
#     phi_tags = re.findall('(\[\*\*.*?\*\*\])', text)
#     for i, tag in enumerate(phi_tags):
#         text = text.replace(tag, '__PHI_{}__'.format(i))
        
    text = re.sub('---+', '\n\n-----\n\n', text)
    text = re.sub('___+', '\n\n_____\n\n', text)
    text = re.sub('[\?\.\!]+(?=[\?\.\!])', '', text)
    text = re.sub('\\+', ' ', text)
    text = re.sub('-|/', '_', text)
    text = re.sub('_+', '_', text)
    text = re.sub('\*|\(|\)', ' ', text)
    text = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', text)
    text = re.sub('\s_\s', ' ', text)
    text = re.sub('\r+|\n+|\u0085+|\u2028+|\u2029+', ' ', text) # Replace newlines
    text = re.sub('\s+|\u00A0+', ' ', text) # Replace multiple spaces w/ single    
    segments = text.split(' ')
    
    return segments

In [3]:
NOTEEVENTS = "../data/NOTEEVENTS.csv"

In [ ]:
notes = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        note = preprocess_text(note[-1])
        notes.append(note)
#         if note[6] == 'Discharge summary':
#             note = preprocess_text(note[-1])
#             notes.append(note)

In [4]:
import gensim
from gensim.models.fasttext import FastText
from timeit import default_timer as timer

In [ ]:
model = FastText(size=300, window=10, min_count=5)

In [ ]:
model.build_vocab(lines)

In [ ]:
print(model)

In [ ]:
start = timer()

model.train(lines, total_examples=len(lines), epochs=5)

end = timer()
print(end - start)

In [ ]:
print(model)

In [ ]:
model.save('saved_model_gensim2')

In [ ]:
model.wv.doesnt_match(['cipro', 'dexamethasone', 'radiology', 'sitagliptin'])

In [ ]:
with open("processed_text.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(notes)

In [7]:
def clean_string1(text):
    """
    """
    text = text.strip().lower().replace('-', '_').replace('.', '_').replace(' ', '_').rstrip('_')
    return text

def preprocess_text2(query):
    """
    """
    query = re.sub('\d+|\d+\.\d+', '[NUM]', query)
    query = re.sub('(\[\*\*.*?\*\*\])', '[PHI]', query)
    query = query.strip('"').strip('?').strip("'").strip('(').strip(')').strip(':')
    query = re.sub('['+'!"#$%&\'()*+,-./:;<=>?@\\^`{|}~'+']', '', query)
    word_list = query.split()
    word_list = [clean_string1(word) for word in word_list]
    return word_list

In [ ]:
notes2 = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        note = preprocess_text2(note[-1])
        notes2.append(note)

In [ ]:
with open("processed_text.csv", "w") as f:
    writer = csv.writer(f)
    writer.writerows(notes2)

In [ ]:
notes2 = []
with open("processed_text.csv", "r") as f:
    reader = csv.reader(f)
    for note in reader:
        notes2.append(note)

In [ ]:
notes2[2]

## FastText model

In [5]:
import gensim
from gensim.models.fasttext import FastText
from gensim.models import Word2Vec
from timeit import default_timer as timer

In [ ]:
ft_model = FastText(size=300, window=10, min_count=5)

In [ ]:
start = timer()

ft_model.build_vocab(notes2)

end = timer()
print(end-start)
print(ft_model)

In [ ]:
start = timer()

ft_model.train(notes2, total_examples=len(notes2), epochs=5)

end = timer()
print(end - start)

In [ ]:
ft_model.save('fasttext_embeddings')

## Word2Vec model

In [ ]:
w2v_model = Word2Vec(size=300, window=10, min_count=5, sg=1)

In [ ]:
start = timer()

w2v_model.build_vocab(notes2)

end = timer()
print(end-start)
print(ft_model)

In [ ]:
start = timer()

w2v_model.train(notes2, total_examples=len(notes2), epochs=5)

end = timer()
print(end - start)

In [ ]:
w2v_model.save('w2v_embeddings')

## Preparing input data/fake labels

In [67]:
word_vectors = Word2Vec.load('w2v_embeddings')
# word_vectors = FastText.load('fasttext_embeddings')

/anaconda2/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
  app.launch_new_instance()


In [71]:
embedding_dim = word_vectors.wv.vectors.shape[1]

In [8]:
input_notes = []
with open(NOTEEVENTS, "r") as file:
    reader = csv.reader(file)
    for i, note in enumerate(reader):
        if note[6] == 'Discharge summary':
            note = preprocess_text2(note[-1])
            input_notes.append(note)

In [29]:
import numpy as np
from sklearn.model_selection import train_test_split

# for local testing subset notes to 5000
subset_notes = input_notes[:5000][:]

N = len(subset_notes)
# Create fake labels
p = [0.4, 0.28, 0.22, 0.19, 0.16]
y = np.array([[np.random.binomial(1, p[i]) for i in range(len(p))] for x in range(N)])
# Create 70/30 train/test split
x_train, x_test, y_train, y_test = train_test_split(subset_notes, y, test_size=0.3)

In [38]:
MAXLEN = 0
for note in subset_notes:
    if len(note) > MAXLEN:
        MAXLEN = len(note)

In [78]:
from keras.preprocessing.text import Tokenizer
max_words = len(w2v_model.wv.vocab)
max_words = 15000
token = Tokenizer(max_words)
token.fit_on_texts(subset_notes)
vocab_size = max_words + 1

sequences = token.texts_to_sequences(x_train)
test_sequences = token.texts_to_sequences(x_test)
## Convert to sequences ##
from keras.preprocessing.sequence import pad_sequences
seq_len = 3000
X = pad_sequences(sequences, maxlen=seq_len)
X_test = pad_sequences(test_sequences, maxlen=seq_len)

In [79]:
embeddings_index = {}
vocab = token.word_index.keys()
for word in vocab:
    if word in w2v_model.wv.vocab:
      coefs = np.asarray(w2v_model.wv[word], dtype='float32')
      embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 46242 word vectors.


In [80]:
word_index = token.word_index
embedding_matrix = np.zeros((vocab_size , embedding_dim))
for word, i in word_index.items():
    if i < vocab_size:
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          # words not found in embedding index will be all-zeros.
          embedding_matrix[i] = embedding_vector

In [ ]:
num_classes = y_test.shape[1]

In [ ]:
from keras.optimizers import Adam
from keras.models import Sequential,Model
from keras.layers import Dense, Activation, Dropout, Embedding, Input, Dropout, Bidirectional, GaussianNoise
from keras.layers.recurrent import LSTM, GRU

## Build the model ##
input = Input(shape=(seq_len,))
x = Embedding(input_dim = vocab_size , output_dim = embedding_dim, weights=[embedding_matrix], trainable=False)(input)
x = GaussianNoise(0.75)(x)
x = Bidirectional(GRU(units = 128, recurrent_dropout=0.2, dropout=0.2, activation = 'relu', return_sequences=True))(x)
x = Bidirectional(GRU(units = 128, recurrent_dropout=0.2, dropout=0.2, activation = 'relu'))(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(num_classes, activation='sigmoid')(x)
model = Model(input,x)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X,y_train,epochs=25, batch_size = 128) 

Epoch 1/25
3500/3500 [==============================] - 2450s 700ms/step - loss: 0.5789 - acc: 0.7272
Epoch 2/25
2944/3500 [========================>.....] - ETA: 6:12 - loss: 0.5616 - acc: 0.7425